In [1]:
!pip install voila
!jupyter serverextension enable voila —sys-prefix

     |████████████████████████████████| 6.9 MB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 505 kB 6.9 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 4.1 MB/s eta 0:00:011
     |████████████████████████████████| 150 kB 7.4 MB/s eta 0:00:01
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Attempting uninstall: nbconvert
    Found existing installation: nbconvert 5.6.1
    Uninstalling nbconvert-5.6.1:
      Successfully uninstalled nbconvert-5.6.1
Enabling: voila
- Writing config: /home/david/.jupyter
    - Validating...
      voila 0.2.2 OK
Enabling: —sys-prefix
- Writing config: /home/david/.jupyter
    - Validating...
Error loading server extension —sys-prefix
      X is —sys-prefix importable?


In [2]:
from fastai.vision.all import *
from fastai.vision.widgets import *
import numpy as np
from skimage.morphology import binary_erosion
import time
import cv2

In [3]:
def get_y():
    return 1

In [4]:
def get_learner(cpu=False):
    fname='export.pkl'
    url='https://www.dropbox.com/s/a2rwhy29wx9s448/export_orig.pkl?dl=1'
    # download_url and file_extract are functions from fastai2
    download_url(url,fname)
    learn = load_learner(fname, cpu=cpu)
    os.remove(fname)
    return learn

In [5]:
def plottingWithRedLabel (x,y):
    #Normalising the x data to 0 to 255 
    x=np.array((x-x.min())/(x.max()-x.min())*255,'uint8')
    #Creating the border of the segmentation
    yEroded=binary_erosion(y,np.ones((5,5)))
    yBorder=np.logical_xor(y,yEroded)
    #Clip the red channel value
    xy=np.clip(x+yBorder*255,0,255)
    return np.stack([x,xy,x], axis=-1)

In [ ]:
learn = get_learner(cpu=True)

In [ ]:
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()

In [ ]:
def on_click(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img)
    start = time.time()
    p = learn.predict(img)  
    mask = cv2.resize(p[1].float().numpy().astype(np.float32), img.shape[:2])
    sugg_contour = plottingWithRedLabel(np.array(img.convert('L')),mask)
    end = time.time()-start
    lbl_pred.value = f'Suggested Contour took {end:01f} seconds to generate'
    with out_pl: display(Image.fromarray(sugg_contour.astype(np.uint8)))

In [ ]:
btn_upload.observe(on_click, names=['data'])

In [ ]:
display(VBox([widgets.Label('Upload a prostate Image:'), btn_upload, out_pl, lbl_pred]))